In [1]:
"""
Model - https://huggingface.co/cognitivecomputations/dolphin-2.2.1-mistral-7b fintuned using LORA on samsum validation set and then quantized to AWQ
dataset finetuned on - samsum validation set

llmsearch example shown on - samsum train set and evaluated on samsum test set

Requires:
nltk==3.8.1
rouge_score==0.1.2
py7zr=0.21.0
exllamav2@https://github.com/turboderp/exllamav2/releases/download/v0.0.14/exllamav2-0.0.14+cu121-cp310-cp310-linux_x86_64.whl

pip install nltk==3.8.1 rouge_score==0.1.2 py7zr==0.21.0 autoawq
"""

import torch
import transformers

import llmsearch
import awq
from awq import AutoAWQForCausalLM

print(awq.__version__,torch.__version__, transformers.__version__, llmsearch.__version__)

/root/miniconda3/envs/llmsearch-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Monkey Patching .generate function of `transformers` library
0.2.4 2.2.0+cu121 4.38.2 0.1.0


In [2]:
import os
from pathlib import Path
from typing import Dict, Any, Optional, Union, List

import nltk
import datasets
import evaluate
import numpy as np

from llmsearch.tuner import Tuner
from llmsearch.utils.mem_utils import gc_cuda
from sklearn.model_selection import GridSearchCV
from llmsearch.utils.common_utils import json_load, json_dump
from llmsearch.utils.model_downloader import download_model_from_hf
from llmsearch.scripts.stopping_criteria import MultiTokenStoppingCriteria
from llmsearch.utils.logging_utils import set_verbosity_info, set_verbosity_debug, set_verbosity_warning
from transformers.modeling_outputs import CausalLMOutputWithPast
from transformers import PreTrainedModel, PretrainedConfig, GenerationConfig, AutoTokenizer, StoppingCriteriaList

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
seed = 42
batch_size = 1
num_tune_samples = 500
num_test_samples = 500

model_id = "Praful932/dolphin-2.2.1-mistral-7b-samsum-ft-v1-awq"
device = "cuda:0"

In [5]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]
    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]
    return preds, labels

def get_rouge_score(y_true: List, y_pred: List):
    preds, gts = postprocess_text(preds=y_pred, labels=[item['summary'] for item in y_true])

    result = rouge_metric.compute(predictions=preds, references=gts, use_stemmer=True, use_aggregator=False)
    return np.mean(result['rouge2'])

class DatasetWrapper:
    def __init__(self, hf_dataset, tokenizer, prompt_template = "Summarize : {dialogue}", input_key = "", output_key = "", system_prompt = "", add_output = True):
        self.tokenizer = tokenizer
        self.hf_dataset = hf_dataset
        self.hf_dataset = self.hf_dataset.map(lambda x : {"chat_format" : ([{'role' : "system", "content" : system_prompt}] if system_prompt else []) + [
            {
                'role' : "user", "content" : prompt_template.format(**{input_key : x[input_key]})
            }
        ] + ([{'role' : 'assistant', "content" : x[output_key]}] if add_output else [])})

    def apply_chat_template(self, add_gen_prompt = True):
        """Converts the dataset to a chat based format"""
        self.hf_dataset = self.hf_dataset.map(lambda x: {"formatted_chat": self.tokenizer.apply_chat_template(x["chat_format"], tokenize=False, add_generation_prompt=add_gen_prompt)})



def load_model_and_tokenizer(model_id, temp_model_dir):
    temp_model_dir.mkdir(exist_ok=True, parents=True)
    output_folder = download_model_from_hf(model_id, save_dir=temp_model_dir, branch="main")

    gc_cuda()

    model = AutoAWQForCausalLM.from_quantized(
        quant_path=output_folder, fuse_layers=True, device_map={"": device}, local_files_only=True
    )
    tokenizer = AutoTokenizer.from_pretrained(
        output_folder, local_files_only=True
    )
    tokenizer.pad_token = tokenizer.unk_token

    return model, tokenizer

def load_dataset():

    # model was finetuned on val set

    train_dataset = datasets.load_dataset("samsum")['train']
    train_dataset = DatasetWrapper(train_dataset, tokenizer, input_key = "dialogue", output_key = "summary", add_output = False)
    train_dataset.apply_chat_template(add_gen_prompt=True)
    train_dataset.hf_dataset = train_dataset.hf_dataset.shuffle(seed=seed)

    samples_to_tune_on = train_dataset.hf_dataset.select(range(num_tune_samples))

    test_dataset = datasets.load_dataset("samsum")['test']
    test_dataset = DatasetWrapper(test_dataset, tokenizer, input_key = "dialogue", output_key = "summary", add_output = False)
    test_dataset.apply_chat_template(add_gen_prompt=True)
    test_dataset.hf_dataset = test_dataset.hf_dataset.shuffle(seed=seed)
    test_samples = test_dataset.hf_dataset.select(range(num_test_samples))

    return samples_to_tune_on, test_samples

In [6]:
# Load Model, Tokenizer, Dataset
temp_model_dir = Path(f"./temp_dir/")
temp_model_dir.mkdir(exist_ok=True, parents=True)

model, tokenizer = load_model_and_tokenizer(model_id, temp_model_dir)

# Dataset we will use to find the best generation parameters and test samples
samples_to_tune_on,test_dataset = load_dataset()

# create stop token criteria
multi_token_stop_criteria_ob = MultiTokenStoppingCriteria(sequence_ids=[32000])
stopping_criteria = StoppingCriteriaList([multi_token_stop_criteria_ob])
callbacks_after_inference = [multi_token_stop_criteria_ob.reset]

rouge_metric = evaluate.load("rouge")

Model already exists in temp_dir/Praful932_dolphin-2.2.1-mistral-7b-samsum-ft-v1-awq. Checking the model files...
Checksum validated: model.safetensors  dbaac51863a65313f57cabca6832ea59fad78b4cc442dd6966479a747ced2a14
[+] Validated checksums of all model files!


Fusing layers...: 100%|██████████| 32/32 [00:02<00:00, 13.17it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
set_verbosity_warning()

In [8]:
tuner_ob = Tuner(
    model=model,
    tokenizer=tokenizer,
    dataset=samples_to_tune_on,
    device="cuda:0",
    batch_size=batch_size,
    tokenizer_encode_args={"padding": "longest",'truncation' : True, "add_special_tokens": False},
    tokenizer_decode_args={"spaces_between_special_tokens": False, 'skip_special_tokens' : True},
    scorer=get_rouge_score,
    prompt_template="{formatted_chat}",
    seed=seed,
    column_mapping={"input_cols": ["formatted_chat"], "eval_cols": ["summary"]},
    callbacks_after_inference=callbacks_after_inference,
)

In [9]:
print(tuner_ob.dataset['_X'][0])

<|im_start|>user
Summarize : Lucy: omg did you see JK this morning?
Sue: I try to avoid it lol
Lucy: you should have seen it it was disgusting
Sue: I cant do it anymore i try to listen to the radio in the mornings.. jk makes you think the whole world is full of idiots lol
Lucy: you may be right I dont know how some of them can go on there in public for the world to see
Sue: I would die if I got a call to go on there lol
Sue: could you imagine ha ha 
Lucy: I would piss myself If I saw you and Andy up there
Sue: over my dead body !<|im_end|>
<|im_start|>assistant



In [10]:
gen_params1 = {
    'max_new_tokens' : 70,
    'stopping_criteria' : stopping_criteria,
    'generation_seed' : 42,
}

scores_before, outputs_before = tuner_ob.get_score(gen_params1)

  0%|          | 0/500 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.
100%|██████████| 500/500 [08:50<00:00,  1.06s/it]


In [11]:
print(scores_before)

0.25437221787091235


In [12]:
hyp_space = {
    'max_new_tokens' : [70],
    'stopping_criteria' : [stopping_criteria],
    'generation_seed' : [42],
    'do_sample' : [True],

    'temperature': [0.1,0.3,0.5,0.7,0.9,1.0],
    'top_k': [50,60,70,80],
    'no_repeat_ngram_size': [0],
}

clf = GridSearchCV(
    estimator = tuner_ob.estimator,
    param_grid=hyp_space,
    scoring = tuner_ob.scorer,
    cv = 2,
    n_jobs = None,
    verbose=3,
)

In [13]:
clf.fit(X=tuner_ob.dataset["_X"], y=tuner_ob.dataset['_y'])

Fitting 2 folds for each of 24 candidates, totalling 48 fits


  2%|▏         | 5/250 [00:05<04:29,  1.10s/it]Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


100%|██████████| 250/250 [03:48<00:00,  1.09it/s]


[CV 1/2] END do_sample=True, generation_seed=42, max_new_tokens=70, no_repeat_ngram_size=0, stopping_criteria=[<llmsearch.scripts.stopping_criteria.MultiTokenStoppingCriteria object at 0x7f3b38303610>], temperature=0.1, top_k=50;, score=0.257 total time= 3.8min


100%|██████████| 250/250 [03:59<00:00,  1.04it/s]


...

100%|██████████| 250/250 [04:19<00:00,  1.04s/it]


[CV 1/2] END do_sample=True, generation_seed=42, max_new_tokens=70, no_repeat_ngram_size=0, stopping_criteria=[<llmsearch.scripts.stopping_criteria.MultiTokenStoppingCriteria object at 0x7f3b38303610>], temperature=1.0, top_k=80;, score=0.143 total time= 4.3min


100%|██████████| 250/250 [04:07<00:00,  1.01it/s]


[CV 2/2] END do_sample=True, generation_seed=42, max_new_tokens=70, no_repeat_ngram_size=0, stopping_criteria=[<llmsearch.scripts.stopping_criteria.MultiTokenStoppingCriteria object at 0x7f3b38303610>], temperature=1.0, top_k=80;, score=0.153 total time= 4.1min


GridSearchCV(cv=2,
             estimator=LLMEstimatorWrapper(batch_size=1,
                                           callbacks_after_inference=[<bound method MultiTokenStoppingCriteria.reset of <llmsearch.scripts.stopping_criteria.MultiTokenStoppingCriteria object at 0x7f3b38303610>>],
                                           device='cuda:0', do_sample=True,
                                           generation_seed=42, is_fitted_=True,
                                           max_new_tokens=70,
                                           model=MistralAWQForCausalLM(
  (model): Mistral...
                                                                  'truncation': True},
                                           top_k=50),
             param_grid={'do_sample': [True], 'generation_seed': [42],
                         'max_new_tokens': [70], 'no_repeat_ngram_size': [0],
                         'stopping_criteria': [[<llmsearch.scripts.stopping_criteria.MultiTokenStoppingCriteria object at 0x7f3b38303610>]],
                         'temperature': [0.1, 0.3, 0.5, 0.7, 0.9, 1.0],
                         'top_k': [50, 60, 70, 80]},
             scoring=make_scorer(get_rouge_score), verbose=3)

In [14]:
scores_after, outputs_after = tuner_ob.get_score(clf.best_params_)

 10%|█         | 52/500 [00:49<07:19,  1.02it/s]Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


100%|██████████| 500/500 [07:43<00:00,  1.08it/s]


In [15]:
print(scores_before, scores_after)

0.25437221787091235 0.25640652169458006


In [16]:
str(clf.best_params_)

"{'do_sample': True, 'generation_seed': 42, 'max_new_tokens': 70, 'no_repeat_ngram_size': 0, 'stopping_criteria': [<llmsearch.scripts.stopping_criteria.MultiTokenStoppingCriteria object at 0x7f3b38303610>], 'temperature': 0.1, 'top_k': 50}"

In [17]:

d = {
    'scores_before' : scores_before,
    'scores_after' : scores_after,
    'outputs_before' : outputs_before,
    'outputs_after' : outputs_after,
    'best_params' : str(clf.best_params_),
}

f = "./samsum-best-params-500s-capybara-7b.json"
json_dump(d, f)

In [18]:
print(d['best_params'])

{'do_sample': True, 'generation_seed': 42, 'max_new_tokens': 70, 'no_repeat_ngram_size': 0, 'stopping_criteria': [<llmsearch.scripts.stopping_criteria.MultiTokenStoppingCriteria object at 0x7f3b38303610>], 'temperature': 0.1, 'top_k': 50}


In [19]:
# harcoding from above file here due to notebook re-run
len(test_dataset)

500

In [20]:
# eval on test samples

gen_params1 = {
    "max_new_tokens": 70,
    "stopping_criteria": stopping_criteria,
    "generation_seed": 42,
}

oos_scores_before, oos_outputs_before = tuner_ob.get_score(gen_params1,test_dataset)

100%|██████████| 500/500 [07:45<00:00,  1.07it/s]


In [21]:
oos_scores_before

0.2554371469507787

In [22]:
test_dataset[1]

{'id': '13681165-1',
 'dialogue': "Alyssa: Have you seen Fergie’s national anthem? Illuminati does a great job.\r\nDerek: This is not normal. I saw it last week…\r\nAlyssa: What do you think about it?\r\nDerek: I can fart bright stripes and bright stars better then she sings.\r\nAlyssa: The best part is that she acts like she nailed it. But at least it's funny in a good way.\r\nDerek: It is 😂",
 'summary': "Derek and Alyssa make fun of Fergie's performance of the national anthem.",
 'chat_format': [{'content': "Summarize : Alyssa: Have you seen Fergie’s national anthem? Illuminati does a great job.\r\nDerek: This is not normal. I saw it last week…\r\nAlyssa: What do you think about it?\r\nDerek: I can fart bright stripes and bright stars better then she sings.\r\nAlyssa: The best part is that she acts like she nailed it. But at least it's funny in a good way.\r\nDerek: It is 😂",
   'role': 'user'}],
 'formatted_chat': "<|im_start|>user\nSummarize : Alyssa: Have you seen Fergie’s nation

In [23]:
oos_scores_after, oos_outputs_after = tuner_ob.get_score(clf.best_params_,test_dataset)

  0%|          | 0/500 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.
100%|██████████| 500/500 [07:50<00:00,  1.06it/s]


In [24]:
print(oos_scores_before, oos_scores_after)

0.2554371469507787 0.2590378656058853


In [26]:
d = {
    'scores_before' : scores_before,
    'scores_after' : scores_after,
    'outputs_before' : outputs_before,
    'outputs_after' : outputs_after,

    'oos_scores_before' : oos_scores_before,
    'oos_scores_after' : oos_scores_after,
    'oos_outputs_before' : oos_outputs_before,
    'oos_outputs_after' : oos_outputs_after,
    'best_params' : str(clf.best_params_),
}

f = "./samsum-best-params-500s-tune-capybara-7b.json"
json_dump(d, f)